In [ ]:
#| hide
from netatmo_thermostat.core import *
from dotenv import load_dotenv
from time import time

In [ ]:
#|  hide
load_dotenv()

True

# netatmo-energy

> A simple Python SDK for the Netatmo Energy API (thermostats).

## Setup

1. Create an app at [dev.netatmo.com](https://dev.netatmo.com) to get your API credentials
2. Either create a `.env` file:
```
CLIENT_ID=your_client_id
CLIENT_SECRET=your_client_secret
ACCESS_TOKEN=your_access_token
REFRESH_TOKEN=your_refresh_token
```
of pass in the vars to `Thermostat(client_id, client_secret, access_token, refresh_token)` when initializing the API client.



## Usage

In [ ]:
t = Thermostat()
hd = t.homesdata()
home_id = hd.homes[0].id
room_id = hd.homes[0].rooms[0].id

# Get current status
t.room_temperatures(home_id)


[{'room_id': '3352988001',
  'temperature': 21.7,
  'setpoint': 21.5,
  'setpoint_mode': 'manual'}]

In [ ]:
# Set room to 20°C for 1 hour
t.setroomthermpoint(home_id, room_id, mode='manual', temp=21, endtime=int(time())+3600)

```python
{'status': 'ok', 'time_server': 1767753063}
```

In [ ]:
t.room_temperatures(home_id)

[{'room_id': '3352988001',
  'temperature': 21.7,
  'setpoint': 21,
  'setpoint_mode': 'manual'}]

In [ ]:
# Get temperature history
r = t.getroommeasure(home_id, room_id)[0]
list(r), r['value'][:3]

(['beg_time', 'step_time', 'value'], [[22.2], [22.2], [22.1]])

## Methods

- `homesdata()` — get homes and topology
- `homestatus(home_id)` — current device status
- `room_temperatures(home_id)` — quick view of all room temps
- `getroommeasure(home_id, room_id, ...)` — temperature history
- `setroomthermpoint(home_id, room_id, mode, ...)` — set room temperature
- `setthermmode(home_id, mode)` — set home mode (schedule/away/hg)
- `getmeasure(device_id, ...)` — boiler history
- `createnewhomeschedule(...)` — create weekly schedule
- `synchomeschedule(...)` — modify schedule
- `switchhomeschedule(home_id, schedule_id)` — activate schedule

## Thermostat Widget

The library includes a ready-to-use FastHTML/MonsterUI thermostat widget for building web dashboards. Use `setup_thermostat_widget()` to register the routes and get the widget component:

```python
from fasthtml.common import fast_app
from netatmo_thermostat import Thermostat, setup_thermostat_widget

app, rt = fast_app()
t = Thermostat()

# Get your home and room IDs
homes = t.homesdata()
home_id = homes.homes[0].id
room_id = homes.homes[0].rooms[0].id

# Setup widget (registers /setpoint route and returns the component)
climate_widget = setup_thermostat_widget(rt, t, home_id, room_id)

@rt("/")
def get():
    return climate_widget
```

![Thermostat Widget](widget-demo.png)

The widget displays current temperature, target setpoint with +/- controls, and a temperature history chart. Clicking the controls makes HTMX requests to adjust the thermostat in real-time.

## Background

Inspired by [Andrej Karpathy's tweet](https://x.com/karpathy) about using Claude Code for home automation. Instead of burning tokens on network scanning, this takes the "boring" approach: read the docs, write a simple SDK.